# Export Table Relational Information as a HTML

In [ ]:
# Install the required libraries for newly created environment - only once

#!pip install mysql-connector-python
#!pip install pandas

In [ ]:
# Import required libraries
import os
import json
import mysql.connector
import pandas as pd

In [ ]:
from bs4 import BeautifulSoup
from getpass import getpass
from mysql.connector import connect, Error

In [ ]:
# Configure username, password and database details
db_user = "<<user_name>>"
db_password = "<<user_password""
bd_host = "<<host_name>>"
db_port = "<<port>>"
db_name = "<<database_name>>"

In [ ]:
# Function to get database connection
def _get_db_connection(db_host, db_port, db_user, db_password, db_name):
    try:
        cnx = connect(
            host=db_host,
            port=db_port,
            user=db_user,
            password=db_password, 
            database=db_name)
        return cnx
    except Error as e:
        print(f"Error: {e}")
        return None

In [ ]:
# Validate database connection
cnx = _get_db_connection(db_host, db_port, db_user, db_password, db_name)
if cnx is not None:
    print("Database connection - successful")
else:
    print("Database connection - failure")

In [ ]:
def _get_html_template():
    """
    :return HTML Template
    """
    _html_template = '''
<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8">
        <title>Table Relations</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
            }}
            .table {{
                border-collapse: collapse;
                width: 80%;
                margin: 20px auto;
            }}
            .table, th, td {{
                border: 1px solid #dddddd;
                text-align: left;
                padding: 8px;
            }}
            th {{
                background-color: #f2f2f2;
            }}
            tr:nth-child(even) {{
                background-color: #f2f2f2;
            }}
            .relation-title {{
                font-weight: bold;
                margin-top: 20px;
            }}
        </style>
    </head>
    <body>
        <h1>Table Relations</h1>
        {relations}
    </body>
</html>
'''
    return _html_template

# Generate Relation

In [ ]:
def _prepare_table_relations():
    """
    :returns Table relational information
    """
    show_db_query = "SHOW TABLES"
    table_relations={}
    try:
        cnx = _get_db_connection(db_host, db_port, db_user, db_password, db_name)
        with cnx.cursor(buffered=True) as cursor:
            cursor.execute(show_db_query)
            tables = cursor.fetchall()
            schema_tables={}
            for table in tables:
                table_name = table[0]
                describe_table=f'DESCRIBE {table_name}'
                cursor.execute(describe_table)
                column_rows = cursor.fetchall()            
                columns=[]
                pricolumns=[]
                for column_row in column_rows:
                    column_row_lower=[str(x).lower() for x in column_row]
                    columns.append(column_row_lower)
                    if column_row_lower[3] == 'pri':
                        pricolumns.append(column_row_lower)
                schema_tables[table_name]={
                    'columns':columns,
                    'pricolumns': pricolumns, 
                }        
            print("Schema collection processing done")
 
            for key in schema_tables.keys():
                table = schema_tables[key]
                columns = table['columns']
                pricolumns = table['pricolumns']

                relations={}
                for pricolumn in pricolumns:
                    r_tables=[]
                    for i_key in schema_tables.keys():
                        if key != i_key:
                            i_table = schema_tables[i_key]
                            for i_column in i_table['columns']:
                                if pricolumn[0] == i_column[0]:
                                    r_tables.append(i_key)
                    if len(r_tables) > 0:
                        relations[pricolumn[0]]=r_tables

                if len(relations) > 0:
                    table_relations[key]=relations    

        cursor.close()
        cnx.close()
    except Error as e:
        print(e)
    return table_relations

In [ ]:
def _create_file(html):
    file_name=f"generated/{db_name}_table_relations.html"
    if not os.path.exists("generated"):
        os.makedirs("generated")
    with open(file_name, 'w') as f:
        f.write(BeautifulSoup(html, 'html.parser').prettify())

In [ ]:
def _export_table_relations_as_html(table_relations):
    relational_table = '<table><th>Table Name</th><th>Relational Column</th><th>Relational Tables</th>'
    
    for table_name, related_tables in table_relations.items():
        for rel_column, rel_tables in related_tables.items():
            rel_table_list = ''.join([f"<li>{r_table}</li>" for r_table in rel_tables])
            relational_table += f"<tr><td>{table_name}</td><td>{rel_column}</td><td><ul>{rel_table_list}</ul></td></tr>"
    
    relational_table += '</table>'
    html = _get_html_template().format(relations=relational_table)
    
    # Create File (Assuming db_name is defined somewhere in your code)
    _create_file(html)

In [ ]:
if __name__ == "__main__":
    print("Export Table Relational Information in HTML format - Started ...")
    table_relations = _prepare_table_relations_to_be_removed()
    _export_table_relations_as_html(table_relations)
    print("Export Table Relational Information in HTML format - Ended - Successfully")